In [ ]:
!pip install -U spacy
!pip install spacy_transformers

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [3]:
spacy.__version__

'3.7.3'

In [4]:
cv_data = json.load(open('/content/drive/MyDrive/ColabNotebooks/Resume parser/train_data.json'))

In [5]:
len(cv_data)

200

In [6]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [8]:
!python -m spacy init fill-config /content/drive/MyDrive/ColabNotebooks/ResumeParser/base_config.cfg /content/drive/MyDrive/ColabNotebooks/ResumeParser/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/ColabNotebooks/ResumeParser/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
def get_spacy_doc(file, data):
  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start,end]) + "   " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)


    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [12]:
len(train), len(test)

(140, 60)

In [14]:
file = open('/content/drive/MyDrive/ColabNotebooks/ResumeParser/train_file.txt', 'w')
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/ColabNotebooks/ResumeParser/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/ColabNotebooks/ResumeParser/test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 81.49it/s]


In [16]:
!python -m spacy train /content/drive/MyDrive/ColabNotebooks/ResumeParser/config.cfg --output /content/drive/MyDrive/ColabNotebooks/ResumeParser/output --paths.train /content/drive/MyDrive/ColabNotebooks/ResumeParser/train_data.spacy --paths.dev /content/drive/MyDrive/ColabNotebooks/ResumeParser/test_data.spacy --gpu-id 0

✔ Created output directory:
/content/drive/MyDrive/ColabNotebooks/ResumeParser/output
ℹ Saving to output directory:
/content/drive/MyDrive/ColabNotebooks/ResumeParser/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
config.json: 100% 481/481 [00:00<00:00, 3.29MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 23.1MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 58.5MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 28.0MB/s]
model.safetensors: 100% 499M/499M [00:02<00:00, 246MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #      

In [18]:
nlp = spacy.load('/content/drive/MyDrive/ColabNotebooks/ResumeParser/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [19]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 48.8 MB/s eta 0:00:00


In [23]:
import sys, fitz

fname = "/content/drive/MyDrive/ColabNotebooks/ResumeParser/test1.pdf"
doc = fitz.open(fname)

In [24]:
text = "  "
for page in doc:
  text = text + str(page.get_text())


In [25]:
text

'  GAURAV CHAUHAN\ngauravchauhan2261@gmail.com | +91-9811665425 | HTTPS://GITHUB.COM/GAURAV0369 | https://www.linkedin.com/in/gauravch0369/ | https://gaurav-chauhan.in/\nSkills\nC | Java | Python | Pandas | Data Analysis | Machine Learning |Tensorflow | Astropy | OpenCV | SQL | Git\nExperience\nGirlScript Summer of Code | Open source contributor\nMay’23 - Aug’23\n● Pull request merged into the WORLD-OF-AI repo by CognitiveLab.\n● Garbage Classiﬁer: custom computer vision algorithm can classify garbage into 7 different kinds of materials,\nincluding organic, cardboard, glass, metal, plastic etc.\n● Tech stack : python, OpenCV, Tensorﬂow\nhttps://github.com/CognitiveLab-tech/World-of-AI/tree/main/deep_learning/Garbage-Classiﬁer\nEducation\nBTech in Computer science with Data science (ML&AI) by Upgrad\n2020- 2024\nMAHARISHI UNIVERSITY OF INFORMATION TECHNOLOGY, NOIDA\nHackathon Projects\nRiddler AI\nMar’23\nBrewing codes 23 by Upgrad and Muit Noida\n● A voice assistant study buddy, which 

In [26]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "   ->>>>   ", ent.label_)

GAURAV CHAUHAN    ->>>>    Name
+91    ->>>>    Email Address
-    ->>>>    Email Address
9811665425    ->>>>    Email Address
BTech in Computer science with Data science (ML&AI)    ->>>>    Degree
2024    ->>>>    Graduation Year
MAHARISHI UNIVERSITY OF INFORMATION TECHNOLOGY    ->>>>    College Name
